In [488]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

### Data Pre-pocssing: 

#### Service County & Walkability: 

In [79]:
# Counties served by Cali Health: 12 counties in total
Service_Area = ['Glenn', 'Butte', 'Colusa', 'Yolo', 'Solano', 'Sacramento', 'El Dorado', 'Sutter', 'Yuba', 'Placer', 'Nevada', 'Sierra']
# Walkability by counties: 12 rows
Service_Area_Long = [County + ' County' for County in Service_Area]
Walkability = pd.read_csv('CountyWalkability.csv')
Walkability = Walkability[Walkability['COUNTYNAME'].isin(Service_Area_Long)]

#### Breakdwon to Zip Code: 

In [ ]:
# Zip codes in service area: 177 rows
Zip = pd.read_csv('CaliforniaZipCodeCounties.csv', index_col=0)
Zip = Zip[Zip['county'].isin(Service_Area)]

#### Census on Zip Code: 

In [300]:
# Census records: 177 rows
Census = pd.read_csv('CaliforniaCensusDataByZip.csv', index_col=0)
Census = Census[Census['Geographic Area Name'].isin(Zip['zip'])]#### Census (Zip Code Level): 

# Reorganize Census data columns into multi-index
Census_Col = [kwd.split(sep="!!") for kwd in Census.columns]
Prim_Label, Seco_Label, Tert_Label = [], [], []
for keywords in Census_Col: 
    Prim_Label.append(keywords[0])
    try: 
        Seco_Label.append(keywords[1])
        try: 
            Tert_Label.append(keywords[-1])
        except IndexError: 
            Tert_Label.append(keywords[1])
    except IndexError: 
        Seco_Label.append(keywords[0])
        Tert_Label.append(keywords[0])

Header = [Prim_Label, Seco_Label, Tert_Label]
Census_Reorg = Census.set_axis(Header, axis=1)

In [296]:
Tert_Label

['Geography',
 'Geographic Area Name',
 'Total population',
 'Male',
 'Female',
 'Sex ratio (males per 100 females)',
 'Under 5 years',
 '5 to 9 years',
 '10 to 14 years',
 '15 to 19 years',
 '20 to 24 years',
 '25 to 34 years',
 '35 to 44 years',
 '45 to 54 years',
 '55 to 59 years',
 '60 to 64 years',
 '65 to 74 years',
 '75 to 84 years',
 '85 years and over',
 'Median age (years)',
 'Under 18 years',
 '16 years and over',
 '18 years and over',
 '21 years and over',
 '62 years and over',
 '65 years and over',
 '18 years and over.1',
 'Male',
 'Female',
 'Sex ratio (males per 100 females)',
 '65 years and over.1',
 'Male',
 'Female',
 'Sex ratio (males per 100 females)',
 'Total population',
 'One race',
 'Two or more races',
 'One race.1',
 'White',
 'Black or African American',
 'American Indian and Alaska Native',
 'Cherokee tribal grouping',
 'Chippewa tribal grouping',
 'Navajo tribal grouping',
 'Sioux tribal grouping',
 'Asian',
 'Asian Indian',
 'Chinese',
 'Filipino',
 'Japan

In [302]:
Census_Reorg.head().T.to_excel('Census_Reorg.xlsx')

In [214]:
Census_Reorg['SEX AND AGE']

Total population                                                  \
     Total population   Male Female Sex ratio (males per 100 females)   
1016            28297  13916  14381                              96.8   
1034            78695  38214  40481                              94.4   
1036             3407   1800   1607                               112   
1065            10330   4836   5494                                88   
1077            29351  14520  14831                              97.9   
...               ...    ...    ...                               ...   
1762             2664   1201   1463                              82.1   
1763             1096    588    508                             115.7   
1764              679    349    330                             105.8   
1765            30369  15865  14504                             109.4   
1767            18782   9504   9278                             102.4   

                                                                              \
     Under 5 years 5 to 9 years 10 to 14 years 15 to 19 years 20 to 24 years   
1016          1482         1529           1725           1547           1067   
1034          5956         4902           5847           5136           5609   
1036           565          439            263            148            478   
1065           158          308            130            479            356   
1077          1743         1941           2114           1925           2031   
...            ...          ...            ...            ...            ...   
1762           274          120            181             19             41   
1763            53           79             41              9             39   
1764            20           36             35              0             32   
1765          1362         1172           1364           1621           2111   
1767          1045         1188           1315            788            865   

                     ...                                      \
     25 to 34 years  ... 62 years and over 65 years and over   
1016           2654  ...              7420              6017   
1034          13344  ...             12279              9916   
1036            984  ...                28                18   
1065            858  ...              5422              4511   
1077           4982  ...              4475              3515   
...             ...  ...               ...               ...   
1762            337  ...               653               557   
1763            290  ...               146               107   
1764            166  ...               193               115   
1765           4999  ...              6525              4992   
1767           1990  ...              3678              2996   

                                                              \
     18 years and over.1 18 years and over 18 years and over   
1016               22499             10821             11678   
1034               58583             28561             30022   
1036                2065              1060              1005   
1065                9331              4394              4937   
1077               22636             10665             11971   
...                  ...               ...               ...   
1762                2070               995              1075   
1763                 914               504               410   
1764                 588               306               282   
1765               25607             13576             12031   
1767               14675              7562              7113   

                                                              \
     18 years and over 65 years and over.1 65 years and over   
1016              92.7                6017              2655   
1034              95.1                9916              4421   
1036             105.5                  18                18   
1065                89      

#### Transit on Zip Code: 

In [ ]:
# Transit stop by Zip code: 177 rows
Transit_Zcta = pd.read_csv('PublicTransitData.csv')
Zcta_Crosswalk = pd.read_excel('ZIPCodetoZCTACrosswalk2022UDS.xlsx')
Transit_A = Transit_Zcta.join(Zcta_Crosswalk.set_index('zcta'), on = 'zcta19', how='left')\
                        .loc[:, ['ZIP_CODE', 'PO_NAME', 'STATE', 'count_ntm_stops', 'totpop13_17']]\
                        .groupby('ZIP_CODE').sum()
Transit_B = Transit_Zcta.join(Zcta_Crosswalk.set_index('zcta'), on = 'zcta19', how='left')\
                        .loc[:, ['ZIP_CODE','stops_per_capita', 'stops_per_sqmile']]\
                        .groupby('ZIP_CODE').mean()
Transit = Transit_A.join(Transit_B, how='left').reset_index()
Transit = Transit[Transit.ZIP_CODE.isin(Zip['zip'])]

#### Vaccine survey: 

In [255]:
Survey = pd.read_csv('VaccinationSurveyData.csv', index_col=0)
Survey = Survey.rename(columns={'I intend to get vaccinated for COVID-19 when the vaccine becomes available to me. (1-7 scale with 7 being Likely and 1 being Unlikely)': 'willingness'})\
                .loc[:, ['Gender', 'Race', 'Hispanic or Latino/A', 'Age', 'willingness']]
Conversion = {
            1: 0.1, 
            2: 0.25, 
            3: 0.4, 
            4: 0.6, 
            5: 0.7, 
            6: 0.85, 
            7: 0.95
            }
Survey['conversion'] = [Conversion[tier] for tier in Survey.willingness]

In [287]:
Survey_Result_Age = Survey[['Age', 'conversion']].groupby(pd.cut(Survey.Age, [0, 20, 25, 35, 45, 55, 60, 65, 75, max(Survey.Age)]), observed=True).mean().loc[:, 'conversion'].to_frame().reset_index()
Survey_Result_Profile = Survey.groupby(['Gender', 'Race', 'Hispanic or Latino/A'], observed=True).mean().loc[:, 'conversion'].to_frame().reset_index()


In [288]:
Survey_Result_Age.sort_values('conversion', ascending=False)

,Age,conversion
7,"(65, 75]",0.758889
5,"(55, 60]",0.752667
0,"(0, 20]",0.747742
4,"(45, 55]",0.747122
1,"(20, 25]",0.738489
2,"(25, 35]",0.729680
3,"(35, 45]",0.722805
6,"(60, 65]",0.690217
8,"(75, 82]",0.633333


In [289]:
Survey_Result_Profile.sort_values('conversion', ascending=False)

,Gender,Race,Hispanic or Latino/A,conversion
17,Non-Binary or Transgender,Asian,Not Hispanic or Latino/A,0.885714
0,Female,Asian,Hispanic or Latino/A,0.858571
8,Male,Asian,Hispanic or Latino/A,0.825676
21,Non-Binary or Transgender,White,Not Hispanic or Latino/A,0.805769
1,Female,Asian,Not Hispanic or Latino/A,0.798558
12,Male,Other,Hispanic or Latino/A,0.776471
6,Female,White,Hispanic or Latino/A,0.771479
14,Male,White,Hispanic or Latino/A,0.768421
15,Male,White,Not Hispanic or Latino/A,0.757023
5,Female,Other,Not Hispanic or Latino/A,0.752778


### Vaccine Demand Estimation: 

In [356]:
# Re-organize census data: 
# Extract ZIP: 
Pop_Zip = Census_Reorg.loc[:, ('Geographic Area Name', 'Geographic Area Name')]
# Extrat basic profile: age and sex
Pop_Basic = Census_Reorg.loc[:, ('SEX AND AGE', 'Total population')].iloc[:, 0:3]
Pop_Age = Census_Reorg.loc[:, ('SEX AND AGE', 'Total population')].iloc[:, 4:17]
Pop_Basic['Non-Binary or Transgender'] = Pop_Basic['Total population'] - Pop_Basic.drop('Total population', axis=1).sum(axis=1)
#Pop_Basic.drop('Total population', axis=1, inplace=True) Retain total population
# Extract race information
Pop_Race = Census_Reorg.loc[:, ('RACE', 'Total population')].loc[:, ['Total population', 'Asian', 'White', 'Black or African American']]
Pop_Race['Other'] = Pop_Race['Total population'] - Pop_Race.drop('Total population', axis=1).sum(axis=1)
Pop_Race.drop('Total population', axis=1, inplace=True)
# Extract hispano information
Pop_His = Census_Reorg.loc[:, ('HISPANIC OR LATINO AND RACE', 'Total population')].iloc[:, 0:2].rename(columns={'Hispanic or Latino (of any race)': "Hispanic or Latino/A"})
Pop_His['Not Hispanic or Latino/A'] = Pop_His['Total population'] - Pop_His['Hispanic or Latino/A']
Pop_His.drop('Total population', axis=1, inplace=True)
# Regroup census data together: 
Census_Extracted = pd.concat([Pop_Zip, Pop_Basic, Pop_Race, Pop_His], axis=1)

/var/folders/y7/c980x4qd7flfptjj2fjrnn0r0000gn/T/ipykernel_1971/4028183379.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  Pop_Zip = Census_Reorg.loc[:, ('Geographic Area Name', 'Geographic Area Name')]
/var/folders/y7/c980x4qd7flfptjj2fjrnn0r0000gn/T/ipykernel_1971/4028183379.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  Pop_Basic = Census_Reorg.loc[:, ('SEX AND AGE', 'Total population')].iloc[:, 0:3]
/var/folders/y7/c980x4qd7flfptjj2fjrnn0r0000gn/T/ipykernel_1971/4028183379.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  Pop_Age = Census_Reorg.loc[:, ('SEX AND AGE', 'Total population')].iloc[:, 4:17]
/var/folders/y7/c980x4qd7flfptjj2fjrnn0r0000gn/T/ipykernel_1971/4028183379.py:10: PerformanceWarning: indexing past lexsort depth may impact performance.
  Pop_Race = Census_Reorg.loc[:, ('RACE', 'Total population')].loc[:, ['Total population', 'Asian', 'White', 'Black or African American']]

In [371]:
# Calculate the ratio
Census_Ratio = pd.concat([Census_Extracted[['Geographic Area Name', 'Total population']], Census_Extracted.iloc[:, 2:].div(Census_Extracted['Total population'], axis=0)], axis=1)
Census_Ratio.head()

,Geographic Area Name,Total population,Male,Female,Non-Binary or Transgender,Asian,White,Black or African American,Other,Hispanic or Latino/A,Not Hispanic or Latino/A
1016,94510,28297,0.491784,0.508216,0.0,0.114641,0.724953,0.032088,0.128317,0.137894,0.862106
1034,94533,78695,0.485596,0.514404,0.0,0.139907,0.436673,0.183341,0.240079,0.355613,0.644387
1036,94535,3407,0.528324,0.471676,0.0,0.049310,0.669797,0.119460,0.161432,0.156736,0.843264
1065,94571,10330,0.468151,0.531849,0.0,0.044240,0.782478,0.109100,0.064182,0.228170,0.771830
1077,94585,29351,0.494702,0.505298,0.0,0.187353,0.394740,0.200981,0.216926,0.272563,0.727437


In [373]:
Survey_Result_Profile.head()

,Gender,Race,Hispanic or Latino/A,conversion
0,Female,Asian,Hispanic or Latino/A,0.858571
1,Female,Asian,Not Hispanic or Latino/A,0.798558
2,Female,Black or African American,Hispanic or Latino/A,0.637179
3,Female,Black or African American,Not Hispanic or Latino/A,0.614348
4,Female,Other,Hispanic or Latino/A,0.705000


In [423]:
Census_Ratio.head()

,Geographic Area Name,Total population,Male,Female,Non-Binary or Transgender,Asian,White,Black or African American,Other,Hispanic or Latino/A,Not Hispanic or Latino/A,demand
1016,94510,28297,0.491784,0.508216,0.0,0.114641,0.724953,0.032088,0.128317,0.137894,0.862106,28297.0
1034,94533,78695,0.485596,0.514404,0.0,0.139907,0.436673,0.183341,0.240079,0.355613,0.644387,78695.0
1036,94535,3407,0.528324,0.471676,0.0,0.049310,0.669797,0.119460,0.161432,0.156736,0.843264,3407.0
1065,94571,10330,0.468151,0.531849,0.0,0.044240,0.782478,0.109100,0.064182,0.228170,0.771830,10330.0
1077,94585,29351,0.494702,0.505298,0.0,0.187353,0.394740,0.200981,0.216926,0.272563,0.727437,29351.0


*We assume that racial distribution is SIMILAR ACROSS ALL GENDER GROUPS:*

In [509]:
# We assume that racial distribution is SIMILAR ACROSS ALL GENDER GROUPS: 
Demand = []
Zip_Demand_Template = {}
for index, zip in Census_Ratio.iterrows(): 
    Zip_Demand_Template[zip['Geographic Area Name']] = Survey_Result_Profile.copy(deep=True)
    Survey_Result_Profile['demand'] = [zip['Total population']*zip[row.Gender] * zip[row.Race] * zip[row['Hispanic or Latino/A']] * row.conversion for i, row in Zip_Demand_Template[zip['Geographic Area Name']].iterrows()]
    Demand.append(Survey_Result_Profile.demand.sum())
Census_Ratio['demand'] = Demand

In [512]:
Census_Ratio.demand.sum()

2385597.8363519404

In [513]:
Census_Ratio['Total population'].sum()

3234671

In [514]:
Census_Ratio.head()

,Geographic Area Name,Total population,Male,Female,Non-Binary or Transgender,Asian,White,Black or African American,Other,Hispanic or Latino/A,Not Hispanic or Latino/A,demand
1016,94510,28297,0.491784,0.508216,0.0,0.114641,0.724953,0.032088,0.128317,0.137894,0.862106,20942.886802
1034,94533,78695,0.485596,0.514404,0.0,0.139907,0.436673,0.183341,0.240079,0.355613,0.644387,57004.179465
1036,94535,3407,0.528324,0.471676,0.0,0.049310,0.669797,0.119460,0.161432,0.156736,0.843264,2473.928838
1065,94571,10330,0.468151,0.531849,0.0,0.044240,0.782478,0.109100,0.064182,0.228170,0.771830,7572.227725
1077,94585,29351,0.494702,0.505298,0.0,0.187353,0.394740,0.200981,0.216926,0.272563,0.727437,21188.342031


3234671 doses required

### Current clinic layout: 

In [515]:
# Current capacity estimation
path = 'CaliforniaCensusDataByZip V1 Renamed.xlsx'
Cali_Facilities = pd.read_excel(path, sheet_name='CaliHealthFacilities')\
                    .loc[:, ['zip', 'county', 'city', 'type', 'pop', 'daily throughput','days to completion']]
Current_Popups = pd.read_excel(path, sheet_name='Pop_up_Clinics')
Current_Popups = Current_Popups.join(Zip[['zip', 'county']].set_index('zip'), on='zip', how='left')\
                    .loc[:, ['zip', 'county', 'city', 'type', 'pop', 'daily throughput','days to completion']]
# Master list of current & proposed facilities
All_Curent_Facilities = pd.concat([Cali_Facilities, Current_Popups], axis=0).drop('days to completion', axis=1)

In [516]:
# Match demand with current list of facilities: 
Zip_Demand = Census_Ratio[['Geographic Area Name', 'demand']].rename(columns={'Geographic Area Name': "zip"})
All_Curent_Facilities = All_Curent_Facilities.join(Zip_Demand.set_index('zip'), on = 'zip', how = 'right')
All_Curent_Facilities['days to completion'] = All_Curent_Facilities.demand / All_Curent_Facilities['daily throughput']

In [517]:
All_Curent_Facilities.sort_values('days to completion').head()

,zip,county,city,type,pop,daily throughput,demand,days to completion
58.0,95651,El Dorado,Lotus,outpatient clinic,480.0,520.0,352.227234,0.677360
100.0,95925,Yuba,Challenge,Pop-Up Clinic,261.0,160.0,188.554045,1.178463
99.0,95736,Placer,Weimar,Pop-Up Clinic,277.0,160.0,202.108154,1.263176
98.0,95606,Yolo,Brooks,Pop-Up Clinic,303.0,160.0,223.130032,1.394563
97.0,95607,Yolo,Capay,Pop-Up Clinic,303.0,160.0,224.498928,1.403118


### Findings: 
- Total 2,385,598 doses required
- Total 16 regions (cities) facing supply issues (completion day > 100 days), 7 coming from Sacramento county
- Toal 37 regions (cities) facing supply leftover (completion day < 5 days), mostly from Placer county (8)
- If release of standard to completion within 10 days, then 11 regions from placer will be anbudent


In [532]:
Zip_Insufficient = All_Curent_Facilities[All_Curent_Facilities['days to completion']>100].sort_values('days to completion', ascending=False)
Counties_Insufficient = Zip_Insufficient.groupby('county').count().loc[:, 'city']
Counties_Insufficient

county
Butte         2
El Dorado     1
Placer        3
Sacramento    5
Solano        2
Yolo          1
Yuba          2
Name: city, dtype: int64

In [544]:
Zip_Insufficient.sort_values('days to completion', ascending=False)

,zip,county,city,type,pop,daily throughput,demand,days to completion
0.0,95616,Yolo,Davis,Pop-Up Clinic,53805.0,160.0,40124.520694,250.778254
1.0,95648,Placer,Lincoln,Pop-Up Clinic,52777.0,160.0,39240.857715,245.255361
15.0,95823,Sacramento,Sacramento,urgent care,80842.0,260.0,58292.797489,224.203067
2.0,95926,Butte,Chico,Pop-Up Clinic,41390.0,160.0,30615.818725,191.348867
3.0,95660,Sacramento,North Highlands,Pop-Up Clinic,36820.0,160.0,26973.446005,168.584038
4.0,95901,Yuba,Marysville,Pop-Up Clinic,35181.0,160.0,26132.329657,163.327060
6.0,95632,Sacramento,Galt,Pop-Up Clinic,32039.0,160.0,23918.898029,149.493113
5.0,95662,Sacramento,Orangevale,Pop-Up Clinic,32363.0,160.0,23907.906694,149.424417
7.0,96150,El Dorado,South Lake Tahoe,Pop-Up Clinic,30369.0,160.0,22637.923336,141.487021
8.0,95966,Butte,Oroville,Pop-Up Clinic,29444.0,160.0,21780.434741,136.127717


In [534]:
Zip_Anbudent = All_Curent_Facilities[All_Curent_Facilities['days to completion']<5].sort_values('days to completion', ascending=False)
Counties_Anbudent = Zip_Anbudent.groupby('county').count().loc[:, 'city']
Counties_Anbudent

county
Butte          5
Colusa         4
El Dorado      6
Glenn          3
Nevada         3
Placer        11
Sacramento     6
Sierra         2
Solano         1
Sutter         4
Yolo           6
Yuba           6
Name: city, dtype: int64

### Solution: 
- Close down / scale down a selection of sites with surplur supply (priotize those with high transit access / walkability, and redistribut the resource to regions facing issues)
- Scale down pop up clinics: 3 in Yolo, 3 in Placer, 1 in Sutter, Yuba and Sacremento. Refer to `Clinics_to_downsize`
- Divert resources to critically overworked areas (>150 days to complete): Davis, Lincoln, Sacramento City, Chico, North Highalnds, and Marysville. Refer to `Zip_Insufficient`

#### Clinics to be allocated to other areas: 

In [521]:
Zip_Anbudent['county long'] = [name + ' County' for name in Zip_Anbudent.county]
Zip_Anbudent_Exp = Zip_Anbudent.join(Transit[['ZIP_CODE', 'stops_per_sqmile']].set_index('ZIP_CODE'), on='zip', how = 'left')\
                                .join(Walkability[['COUNTYNAME', 'Average Walkability']].set_index('COUNTYNAME'), on='county long', how = 'left')

In [542]:
transit_scaler = np.max(Zip_Anbudent_Exp.stops_per_sqmile) - np.min(Zip_Anbudent_Exp.stops_per_sqmile)
Zip_Anbudent_Exp['transit_rating'] = Zip_Anbudent_Exp.stops_per_sqmile - np.min(Zip_Anbudent_Exp.stops_per_sqmile) / transit_scaler

walk_scaler = np.max(Zip_Anbudent_Exp['Average Walkability']) - np.min(Zip_Anbudent_Exp['Average Walkability'])
Zip_Anbudent_Exp['walkability_rating'] = Zip_Anbudent_Exp['Average Walkability'] - np.min(Zip_Anbudent_Exp['Average Walkability']) / walk_scaler

Zip_Anbudent_Exp['access_rating'] = Zip_Anbudent_Exp['transit_rating'] + Zip_Anbudent_Exp['walkability_rating']
Zip_Anbudent_Exp.sort_values('access_rating', ascending=False).reset_index(drop=True)

# Clinics with demand < 3 days and access rating > 8.3: 
Wip = Zip_Anbudent_Exp[Zip_Anbudent_Exp['days to completion']<3]
Wip = Wip[Wip.type == 'Pop-Up Clinic']
Clinics_to_downsize = Wip[Wip.access_rating>8.3]


In [543]:
Clinics_to_downsize

,zip,county,city,type,pop,daily throughput,demand,days to completion,county long,stops_per_sqmile,Average Walkability,transit_rating,walkability_rating,access_rating
82.0,95652,Sacramento,Mcclellan,Pop-Up Clinic,635.0,160.0,471.284596,2.945529,Sacramento County,4.480939,10.012663,4.480939,8.192610,12.673549
85.0,95668,Sutter,Pleasant Grove,Pop-Up Clinic,588.0,160.0,436.292170,2.726826,Sutter County,0.000000,11.389535,0.000000,9.569482,9.569482
88.0,95653,Yolo,Madison,Pop-Up Clinic,502.0,160.0,377.361046,2.358507,Yolo County,0.193932,10.478699,0.193932,8.658647,8.852578
90.0,96141,Placer,Homewood,Pop-Up Clinic,463.0,160.0,344.559670,2.153498,Placer County,0.000000,10.275592,0.000000,8.455540,8.455540
91.0,95714,Placer,Dutch Flat,Pop-Up Clinic,445.0,160.0,330.108930,2.063181,Placer County,0.000000,10.275592,0.000000,8.455540,8.455540
97.0,95607,Yolo,Capay,Pop-Up Clinic,303.0,160.0,224.498928,1.403118,Yolo County,0.007440,10.478699,0.007440,8.658647,8.666086
98.0,95606,Yolo,Brooks,Pop-Up Clinic,303.0,160.0,223.130032,1.394563,Yolo County,0.027502,10.478699,0.027502,8.658647,8.686149
99.0,95736,Placer,Weimar,Pop-Up Clinic,277.0,160.0,202.108154,1.263176,Placer County,0.000000,10.275592,0.000000,8.455540,8.455540


### Unused codes: 

In [524]:

Facilities_Count = All_Curent_Facilities[['county', 'city', 'type', 'days to completion']].groupby(['county', 'city', 'type']).count()\
                    .rename(columns={'days to completion': 'number of facilities'})

In [525]:
Facilities_Count.reset_index()

,county,city,type,number of facilities
0,Butte,Bangor,Pop-Up Clinic,1
1,Butte,Berry Creek,Pop-Up Clinic,1
2,Butte,Biggs,Pop-Up Clinic,1
3,Butte,Chico,Pop-Up Clinic,1
4,Butte,Durham,Pop-Up Clinic,1
...,...,...,...,...
136,Yuba,Marysville,Pop-Up Clinic,1
137,Yuba,Olivehurst,Pop-Up Clinic,1
138,Yuba,Oregon House,Pop-Up Clinic,1
139,Yuba,Smartville,Pop-Up Clinic,1


In [526]:
Facilities_Count.reset_index().loc[:,['city', 'number of facilities']].groupby('city').sum().sort_values('number of facilities', ascending=False).head(10)

,number of facilities
city,
Sacramento,16
Vallejo,4
Roseville,3
Yuba City,2
West Sacramento,2
Rancho Cordova,2
Vacaville,2
Elk Grove,2
Woodland,2


In [527]:
Current_Popups.head()

,zip,county,city,type,pop,daily throughput,days to completion
0,95616,Yolo,Davis,Pop-Up Clinic,53805,160,336.28125
1,95648,Placer,Lincoln,Pop-Up Clinic,52777,160,329.85625
2,95926,Butte,Chico,Pop-Up Clinic,41390,160,258.68750
3,95660,Sacramento,North Highlands,Pop-Up Clinic,36820,160,230.12500
4,95901,Yuba,Marysville,Pop-Up Clinic,35181,160,219.88125


In [528]:
Covered_Zips = pd.Series(pd.concat([Current_Popups.Zip, Cali_Facilities.Zip], axis=0).unique(), name='zips')
Missed_Zips = Zip[~Zip['zip'].isin(Covered_Zips)]

AttributeError: 'DataFrame' object has no attribute 'Zip'

In [ ]:
Missed_Zips['COUNTYNAME'] = [county + " County" for county in Missed_Zips.county]
Missed_Zips = Missed_Zips.set_index('COUNTYNAME').join(Walkability.set_index('COUNTYNAME').loc[:, 'Average Walkability'], how='left')

/var/folders/y7/c980x4qd7flfptjj2fjrnn0r0000gn/T/ipykernel_1971/2180965576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Missed_Zips['COUNTYNAME'] = [county + " County" for county in Missed_Zips.county]


In [ ]:
Missed_Zips

,zip,city,county,lat,long,Average Walkability
COUNTYNAME,,,,,,
Butte County,95928,Chico,Butte,39.7224,-121.8113,9.166913
Butte County,95973,Chico,Butte,39.8032,-121.8673,9.166913
Butte County,95965,Oroville,Butte,39.6054,-121.5751,9.166913
Nevada County,95945,Grass Valley,Nevada,39.2081,-121.0069,8.497368
Placer County,95677,Rocklin,Placer,38.7877,-121.2366,10.275592
Sacramento County,95828,Sacramento,Sacramento,38.4826,-121.4006,10.012663
Sacramento County,95610,Citrus Heights,Sacramento,38.6946,-121.2692,10.012663
Sacramento County,95822,Sacramento,Sacramento,38.5091,-121.4935,10.012663
Sacramento County,95838,Sacramento,Sacramento,38.6406,-121.4440,10.012663


In [ ]:
Walkability.sort_values('Average Walkability', ascending=False)

,COUNTYNAME,Average Walkability,Std Deviation Walkability,Minimum Walkability Census Tract,25th percentile walkability census tract,50th percentile walkability census tract,75th percentile walkability census tract,Maximum Walkability Census Tract
50,Sutter County,11.389535,4.620551,1,7.333333,12.833333,15.125000,19.833333
56,Yolo County,10.478699,4.082574,1,7.000000,11.000000,13.666667,20.000000
47,Solano County,10.348954,4.244249,1,6.833333,10.833333,13.666667,19.833333
30,Placer County,10.275592,4.430669,1,6.500000,10.500000,13.833333,20.000000
33,Sacramento County,10.012663,4.271696,1,6.333333,10.333333,13.500000,20.000000
8,El Dorado County,9.928559,4.554904,1,6.000000,9.500000,14.000000,19.833333
5,Colusa County,9.363060,4.084583,1,6.000000,9.000000,12.833333,19.333333
3,Butte County,9.166913,4.577870,1,5.333333,8.166667,13.333333,20.000000
45,Sierra County,8.828744,4.113998,1,5.666667,8.166667,12.000000,20.000000
28,Nevada County,8.497368,4.134785,1,5.333333,7.666667,11.666667,19.166667


In [ ]:
Missed_Zips.head()

,zip,city,county,lat,long
132,95828,Sacramento,Sacramento,38.4826,-121.4006
266,95610,Citrus Heights,Sacramento,38.6946,-121.2692
289,95822,Sacramento,Sacramento,38.5091,-121.4935
372,95838,Sacramento,Sacramento,38.6406,-121.4440
385,95835,Sacramento,Sacramento,38.6626,-121.4834
